# Addestramento modelli

In [ ]:
import warnings
import numpy as np
import pandas as pd
import os
import sys
import json

In [ ]:
warnings.filterwarnings("ignore")
sys.path.insert(0, "..")

In [ ]:
import utility

In [ ]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, OneHotEncoder

In [ ]:
import tensorflow as tf
from tensorflow import Tensor 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dropout, Concatenate, Input, GlobalAveragePooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Attention, AdditiveAttention
from tensorflow.keras.layers import Activation, BatchNormalization, Add, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import L1, L2, L1L2

In [ ]:
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.layers.merge import concatenate

In [ ]:
import kerastuner as kt
from kerastuner import Hyperband, BayesianOptimization, RandomSearch, HyperModel

In [ ]:
K.set_epsilon(1)

In [ ]:
model_trained_base_path = "TrainedModels"
dataset_path = os.path.join("..", "Preprocessing", "Dataset", "dataset_training.csv")

### Import dei dati

In [ ]:
df = utility.import_dataset(dataset_path, "2021", "2023")
display(df)

### OneHot encoding e normalizzazione

In [ ]:
df, scaler = utility.processing_dataset(df)

### Splitting in training, validation e test

In [ ]:
df_train, df_validation, df_test = utility.split_dataset(df, "2021-03", "2022-04")

### Model 1 - PAPER0 - LSTMwithoutOtherFeature

In [ ]:
class Model1(HyperModel):
    
    def build(self, hp):
        
        time_steps = 24 * hp.Choice("days", [3, 5, 7], ordered=True)

        dropout_rate = hp.Choice("dropout_rate", [0.1, 0.2, 0.3, 0.4, 0.5])

        regularizer_type = hp.Choice("regularizer_type", ["none", "L1", "L2"])
        regularizer_param = hp.Choice("regularizer_param", values=[1e-1, 1e-2, 1e-3], parent_name="regularizer_type", parent_values=["L1", "L2"])
    
        if regularizer_type == "L1":
            regularizer = L1(regularizer_param)        
        elif regularizer_type == "L2":
            regularizer = L2(regularizer_param)
        elif regularizer_type == "none":
            regularizer = None 

        model = Sequential()
        model.add(Input(shape=(time_steps, 1)))
        model.add(LSTM(units=128, return_sequences=True, activity_regularizer=regularizer))
        model.add(LSTM(units=128, activity_regularizer=regularizer))
        model.add(Dropout(rate=dropout_rate))
        model.add(Dense(units=1, activation="linear"))

        return model

    def fit(self, hp, model, *args, **kwargs):
                
        time_steps = 24 * hp.Choice("days", [3, 5, 7], ordered=True)

        train = utility.windowed_dataset(df_train, time_steps)
        validation = utility.windowed_dataset(df_validation, time_steps)
        
        learning_rate=hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
        
        batch_size = hp.Choice("batch_size", values=[16, 32, 64, 128, 256])
        
        return model.fit(
            utility.get_x(train, "CaricoTotale")[0], utility.get_y(train, "CaricoTotale"), 
            batch_size=batch_size,
            epochs=70,
            validation_data=(utility.get_x(validation, "CaricoTotale")[0], utility.get_y(validation, "CaricoTotale")),
            shuffle=False,
            *args,
            **kwargs,
        )

### Model 2 - PAPER12 - LSTMwithOtherFeature

In [ ]:
class Model2(HyperModel):
    
    def build(self, hp):
        
        time_steps = 24 * hp.Choice("days", [3, 5, 7], ordered=True)
        dim_other_feature = 24 + 7 + 12 + 1

        dropout_rate = hp.Choice("dropout_rate", [0.1, 0.2, 0.3, 0.4, 0.5])

        regularizer_type = hp.Choice("regularizer_type", ["none", "L1", "L2"])
        regularizer_param = hp.Choice("regularizer_param", values=[1e-1, 1e-2, 1e-3], parent_name="regularizer_type", parent_values=["L1", "L2"])

        if regularizer_type == "L1":
            regularizer = L1(regularizer_param)        
        elif regularizer_type == "L2":
            regularizer = L2(regularizer_param)
        elif regularizer_type == "none":
            regularizer = None  

        # Pipeline LOAD
        input_load = Input(shape=(time_steps, 1))
        lstm_layer = LSTM(64, return_sequences=False, activity_regularizer=regularizer)(input_load)
        dropout1 = Dropout(rate=dropout_rate)(lstm_layer)

        # Pipeline OTHER FEATURE
        input_other_feature = Input(shape=(dim_other_feature, ))
        dense1 = Dense(64, activation="relu", activity_regularizer=regularizer)(input_other_feature)
        dropout2 = Dropout(rate=dropout_rate)(dense1)

        # Concatenate PIPLINES
        concatenated_layer = concatenate([dropout1, dropout2], axis=1)
        dense2 = Dense(64, activation="relu", activity_regularizer=regularizer)(concatenated_layer)
        dropout3 = Dropout(rate=dropout_rate)(dense2)
        output_layer = Dense(1, activation="linear")(dropout3)
        model = Model(inputs=[input_load, input_other_feature], outputs=[output_layer])

        return model

    def fit(self, hp, model, *args, **kwargs):
                
        time_steps = 24 * hp.Choice("days", [3, 5, 7], ordered=True)

        train = utility.windowed_dataset(df_train, time_steps)
        validation = utility.windowed_dataset(df_validation, time_steps)
        
        learning_rate=hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
        
        batch_size = hp.Choice("batch_size", values=[16, 32, 64, 128, 256])
        
        return model.fit(
            utility.get_x(train, "CaricoTotale"), utility.get_y(train, "CaricoTotale"), 
            batch_size=batch_size,
            epochs=70,
            validation_data=(utility.get_x(validation, "CaricoTotale"), utility.get_y(validation, "CaricoTotale")),
            shuffle=False,
            *args,
            **kwargs,
        )

### Model 3 - PAPER13 - Conv1D + LSTM

In [ ]:
class Model3(HyperModel):
    
    def build(self, hp):
        
        time_steps = 24 * hp.Choice("days", [3, 5, 7], ordered=True)
        dim_other_feature = 24 + 7 + 12 + 1

        dropout_rate = hp.Choice("dropout_rate", [0.1, 0.2, 0.3, 0.4, 0.5])

        regularizer_type = hp.Choice("regularizer_type", ["none", "L1", "L2"])
        regularizer_param = hp.Choice("regularizer_param", values=[1e-1, 1e-2, 1e-3], parent_name="regularizer_type", parent_values=["L1", "L2"])

        if regularizer_type == "L1":
            regularizer = L1(regularizer_param)        
        elif regularizer_type == "L2":
            regularizer = L2(regularizer_param)
        elif regularizer_type == "none":
            regularizer = None  

        # Pipeline LOAD
        input_load = Input(shape=(time_steps, 1))

        # Pipeline 1
        branch1_1 = Conv1D(filters=64, kernel_size=3, activation="relu", activity_regularizer=regularizer)(input_load)
        branch1_2 = MaxPooling1D(pool_size=2, strides=2)(branch1_1)

        # Pipeline 2
        branch2_1 = Conv1D(filters=64, kernel_size=5, activation="relu", activity_regularizer=regularizer)(input_load)
        branch2_2 = MaxPooling1D(pool_size=2, strides=2)(branch2_1)

        # Pipeline 3
        branch3_1 = Conv1D(filters=64, kernel_size=7, activation="relu", activity_regularizer=regularizer)(input_load)
        branch3_2 = MaxPooling1D(pool_size=2, strides=2)(branch3_1)

        # Concatenate pipelines LOAD
        concatenated_layer1 = concatenate([branch1_2,branch2_2,branch3_2], axis=1)
        dropout0 = Dropout(rate=dropout_rate)(concatenated_layer1)
        lstm_layer = LSTM(64, return_sequences=True, activity_regularizer=regularizer)(dropout0)
        global_average_pooling_layer = GlobalAveragePooling1D()(lstm_layer)

        # Pipeline OTHER FEATURE
        input_other_feature = Input(shape=(dim_other_feature, ))
        dense1 = Dense(64, activation="relu", activity_regularizer=regularizer)(input_other_feature)
        dropout1 = Dropout(rate=dropout_rate)(dense1)
        flatten1 = Flatten()(dropout1)

        # Concatenate PIPLINES
        concatenated_layer2 = concatenate([global_average_pooling_layer,flatten1], axis=1)
        dense2 = Dense(64, activation="relu", activity_regularizer=regularizer)(concatenated_layer2)
        dropout2 = Dropout(rate=dropout_rate)(dense2)
        dense3 = Dense(64, activation="relu", activity_regularizer=regularizer)(dropout2)
        dropout3 = Dropout(rate=dropout_rate)(dense3)
        output_layer = Dense(1, activation="linear")(dropout3)
        model = Model(inputs=[input_load, input_other_feature], outputs=[output_layer])

        return model

    def fit(self, hp, model, *args, **kwargs):
                
        time_steps = 24 * hp.Choice("days", [3, 5, 7], ordered=True, default=5)

        train = utility.windowed_dataset(df_train, time_steps)
        validation = utility.windowed_dataset(df_validation, time_steps)
        
        learning_rate=hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
        
        batch_size = hp.Choice("batch_size", values=[16, 32, 64, 128, 256])
        
        return model.fit(
            utility.get_x(train, "CaricoTotale"), utility.get_y(train, "CaricoTotale"), 
            batch_size=batch_size,
            epochs=70,
            validation_data=(utility.get_x(validation, "CaricoTotale"), utility.get_y(validation, "CaricoTotale")),
            shuffle=False,
            *args,
            **kwargs,
        )

### Model 4 - PAPER18 - Wavenet

In [ ]:
class Model4(HyperModel):
    
    def build(self, hp):
        
        time_steps = 24 * hp.Choice("days", [3, 5, 7], ordered=True)
        dim_other_feature = 24 + 7 + 12 + 1

        dropout_rate = hp.Choice("dropout_rate", [0.1, 0.2, 0.3, 0.4, 0.5])

        regularizer_type = hp.Choice("regularizer_type", ["none", "L1", "L2"])
        regularizer_param = hp.Choice("regularizer_param", values=[1e-1, 1e-2, 1e-3], parent_name="regularizer_type", parent_values=["L1", "L2"])

        if regularizer_type == "L1":
            regularizer = L1(regularizer_param)        
        elif regularizer_type == "L2":
            regularizer = L2(regularizer_param)
        elif regularizer_type == "none":
            regularizer = None  

        # Pipeline LOAD
        input_load = Input(shape=(time_steps, 1))
        x = input_load
        for rate in (1, 2, 4, 8, 16, 32):
            x = Conv1D(
                filters=32, kernel_size=2, strides=1, padding="causal", activation="relu", dilation_rate=rate, 
                activity_regularizer=regularizer
            )(x)
        flatten = Flatten()(x)

        # Pipeline OTHER FEATURE
        input_other_feature = Input(shape=(dim_other_feature, ))

        # Concatenate PIPLINES
        concatenated_layer = concatenate([flatten, input_other_feature], axis=1)
        dropout1 = Dropout(rate=dropout_rate)(concatenated_layer)
        dense = Dense(64, activation="relu", activity_regularizer=regularizer)(dropout1)
        dropout2 = Dropout(rate=dropout_rate)(dense)
        output_layer = Dense(1, activation="linear")(dropout2)
        model = Model(inputs=[input_load, input_other_feature], outputs=[output_layer])

        return model

    def fit(self, hp, model, *args, **kwargs):
                
        time_steps = 24 * hp.Choice("days", [3, 5, 7], ordered=True)

        train = utility.windowed_dataset(df_train, time_steps)
        validation = utility.windowed_dataset(df_validation, time_steps)
        
        learning_rate=hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mape"])
        
        batch_size = hp.Choice("batch_size", values=[16, 32, 64, 128, 256])
        
        return model.fit(
            utility.get_x(train, "CaricoTotale"), utility.get_y(train, "CaricoTotale"), 
            batch_size=batch_size,
            epochs=70,
            validation_data=(utility.get_x(validation, "CaricoTotale"), utility.get_y(validation, "CaricoTotale")),
            shuffle=False,
            *args,
            **kwargs,
        )

### Tuning modello

In [ ]:
directory = "TuningDirectory"
project_name = "model1"
file_name = "hyperparameters.csv"
model = Model1()

In [ ]:
tuner = BayesianOptimization(model, objective="val_mape", max_trials=30, directory=directory, project_name=project_name)
tuner.search(callbacks=[EarlyStopping(monitor="val_mape", patience=5)])
utility.make_tuning_csv(directory, project_name, file_name)

### Visualizzazioni tuning

In [ ]:
import os
import pandas as pd
import sys
sys.path.insert(0, "..")
import utility

In [ ]:
directory = "TuningDirectory"
project_name = "model1"
file_name = "hyperparameters.csv"
df = pd.read_csv(os.path.join(directory, project_name, file_name))
df["MAPE"] = df["MAPE"] - 9

In [ ]:
df_sorted = df.sort_values(by="MAPE")
df_sorted = df_sorted.drop_duplicates(subset=['days', 'batch_size', 'learning_rate', 'dropout_rate', 'regularizer_type', 'regularizer_param'])
df_sorted

In [ ]:
utility.print_df_tuning(df_sorted)